In [ ]:
%matplotlib qt
import math
import numpy as np
import pyxem as pxm
import hyperspy.api as hs

from skimage import morphology, filters

import matplotlib.pyplot as plt

import warnings
# Silence some future warnings and user warnings (float64 -> uint8) in skimage
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [ ]:
data_dir = 'D:/Dokumenter/MTNANO/Prosjektoppgave/SPED_data_GaAs_NW/'
#in_file = data_dir + 'gen/Julie_180510_SCN45_FIB_c_crop.hdf5'
#out_file = data_dir + 'gen/Julie_180510_SCN45_FIB_c_crop_{}.hdf5'
in_file = data_dir + 'raw/Julie_180510_SCN45_FIB_a.blo'
out_file = data_dir + 'gen/Julie_180510_SCN45_FIB_a_{}.hdf5'

In [ ]:
s = pxm.load(in_file, lazy=True)
s = s.inav[90:100, 110:150]
if s.data.dtype != 'float64':
    s.change_dtype('float64')

# Convert to a pyxem ElectronDiffraction, conserve the metadata and add some more
s_metadata = s.metadata
s = pxm.ElectronDiffraction(s)
s.metadata = s_metadata

In [ ]:
s.center_direct_beam(radius_start=2, radius_finish=5)

In [ ]:
def rescale(s):
    s.data *= 1 / np.max(s.data)
    
def mask_direct_beam(s):
    direct_beam_mask_size = 7
    mask = s.get_direct_beam_mask(direct_beam_mask_size)
    return pxm.ElectronDiffraction(s*np.invert(mask))

def filename(base, *args):
    return base.format('_'.join((str(arg) for arg in args)).replace('.', '_'))

Parameter testing is done below

In [ ]:
h = 0.55
s_hdome = s.remove_background('h-dome', h=h)
rescale(s_hdome)
s_hdome = mask_direct_beam(s_hdome)
s_hdome.save(filename(out_file, 'h_dome', 0.55), overwrite=True)

In [ ]:
sigma_min = 1
sigma_max = 6
s_gaussian = s.remove_background('gaussian_difference', sigma_min=sigma_min, sigma_max=sigma_max)
rescale(s_gaussian)
s_gaussian = mask_direct_beam(s_gaussian)
s_gaussian.save(filename(out_file, 'gaussian_difference', sigma_min, sigma_max), overwrite=True)

In [ ]:
footprint = 15
s_median = s.remove_background('median', footprint=footprint)
rescale(s_median)
s_median = mask_direct_beam(s_median)
s_median.save(filename(out_file, 'median', footprint), overwrite=True)

In [ ]:
s_hdome.plot()

## Parameter testing

In [ ]:
sample_image_x = 0
sample_image_y = 0
s_test_area = s.inav[sample_image_x, sample_image_y]

### Laplace edge detection + thresholding

- Gaussian blur to remove rippels. Each pixel gets a new value wich is the weighted average of its neighborhood. The original value is weighted the most, while the weighting gets smaller further away from the original pixel.
- Laplacian of blurred image. The second derivative finds the zero crossing (i.e. the edges).
- Compare zero crossings to threshold to remove zero crossings due to small intensity variations.
- Median filter to remove spot noise. Replaces each pixel value by the median of the neighboring pixel values covered by a window. The window can have different shapes, but the total number of pixels covered by the window should be odd so that the median can be extracted easily. The technique is good for removing "salt and pepper noise", i.e. sudden disturbances seen as sparsely distributed black and white pixels. It also preserves the edges well while the noise is being removed.

TODO: Source for this algorithm?

In [ ]:
gauss_stddev = 0.5
laplace_size = 3
threshold = 0.065
median_neighbourhood = morphology.disk(1)

In [ ]:
image = s.data[0, 0]
image_gaussian_blur = filters.gaussian(image, gauss_stddev)
image_laplace = filters.laplace(image_gaussian_blur, laplace_size)
threshold_mask = np.where(image_laplace > threshold)
image_threshold = np.zeros_like(image_laplace)
image_threshold[threshold_mask] = image_laplace[threshold_mask]
image_median = filters.median(image_threshold, median_neighbourhood)

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, sharex=True, sharey=True, figsize=(8,8))
ax = axes.ravel()
ax[0].imshow(image_gaussian_blur)
ax[0].set_title('Gaussian blur, stddev={}'.format(gauss_stddev))

ax[1].imshow(image_laplace)
ax[1].set_title('Laplacian edge, operator size={}'.format(laplace_size))

ax[2].imshow(image_threshold)
ax[2].set_title('Thresholded > {}'.format(threshold))

ax[3].imshow(image_median)
ax[3].set_title('Median disk filter')

for a in ax:
    a.axis('off')
    
plt.tight_layout()
plt.show()

## Build-in background removal from pyxem

In [ ]:
help(s.remove_background)

In [ ]:
def plot_image_comparison(processed, title, param_min, param_max):
    s = pxm.ElectronDiffraction(processed)
    for i in range(len(param_min)):
        print('{} -> [{}, {}], [0, {}]'.format(i, param_min[i], param_max[i], processed.shape[i]))
        s.axes_manager.navigation_axes[i].calibrate((param_min[i], param_max[i]), (0, processed.shape[i]))
    s.plot(cmap='viridis_r')

### H-dome
Test different values of h_cutoff. 0.3 seems good on this dataset.

In [ ]:
h_min = 0.05
h_max = 0.9
h_step = 0.05
s_hdomes = np.empty((int((h_max - h_min) / h_step), s.data.shape[2], s.data.shape[3]))
for i, h_cutoff in enumerate(np.arange(h_min, h_max, h_step)):
    s_hdomes[i] = s_test_area.remove_background('h-dome', h=h_cutoff, show_progressbar=False)
plot_image_comparison(s_hdomes, 'h-dome bg rem', [h_min], [h_max])

### Gaussian
$\sigma_{\text{max}} = 6$, $\sigma_{\text{min}} = 1$ gives reasonable results.

In [ ]:
gauss_stddev_max_min = 2
gauss_stddev_max_max = 15
gauss_stddev_max_step = 0.2
gauss_stddev_min_min = 1
gauss_stddev_min_max = 4
gauss_stddev_min_step = 0.2
gauss_stddev_maxs = np.arange(gauss_stddev_max_min, gauss_stddev_max_max, gauss_stddev_max_step)
gauss_stddev_mins = np.arange(gauss_stddev_min_min, gauss_stddev_min_max, gauss_stddev_min_step)
s_gaussians = np.empty((
    len(gauss_stddev_maxs),
    len(gauss_stddev_mins),
    s.data.shape[2], s.data.shape[3]))

for i, gauss_stddev_max in enumerate(gauss_stddev_maxs):
    for j, gauss_stddev_min in enumerate(gauss_stddev_mins):
        s_gaussians[i, j] = s_test_area.remove_background('gaussian_difference',
                                                          sigma_min=gauss_stddev_min, sigma_max=gauss_stddev_max,
                                                          show_progressbar=False)
        
plot_image_comparison(s_gaussians, 'Gaussian difference bg rem',
              [gauss_stddev_max_min, gauss_stddev_min_min],
              [gauss_stddev_max_max, gauss_stddev_min_max])

### Median
A footprint of 15 gives good results (documentation suggests 'Should be large enough that it is about 3x as big as the size of the peaks').

In [ ]:
median_implementation = 'scipy'
median_footprint_min = 8
median_footprint_max = 50
median_footprint_step = 1
median_footprints = np.arange(median_footprint_min, median_footprint_max, median_footprint_step)
s_medians = np.empty((len(median_footprints), s.data.shape[2], s.data.shape[3]))
for i, footprint in enumerate(median_footprints):
    s_medians[i] = s_test_area.remove_background('median',
                                                 footprint=footprint,
                                                 implementation=median_implementation,
                                                 show_progressbar=False)
plot_image_comparison(s_medians, 'Median filter bg rem',
                     [median_footprint_min], [median_footprint_max])